Change the path to where you would like to save the output file

In [4]:
filepath = "C:/Users/abraks/Documents/CS 548/ARCH5/human_matrix.h5"

install package BiocManager (and other dependencies if you're missing them

In [2]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.10")

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Old packages: 'askpass', 'backports', 'BH', 'boot', 'broom', 'callr', 'caret',
  'class', 'cli', 'clipr', 'cluster', 'curl', 'data.table', 'DBI', 'dbplyr',
  'digest', 'dplyr', 'ellipsis', 'evaluate', 'fansi', 'forcats', 'foreach',
  'foreign', 'formatR', 'fs', 'ggplot2', 'glmnet', 'glue', 'gower', 'haven',
  'hexbin', 'hms', 'htmltools', 'htmlwidgets', 'httpuv', 'httr', 'ipred',
  'IRkernel', 'iterators', 'jsonlite', 'KernSmooth', 'knitr', 'later',
  'lattice', 'lava', 'lubridate', 'markdown', 'MASS', 'Matrix', 'mgcv', 'mime',
  'ModelMetrics', 'modelr', 'nlme', 'nnet', 'numDeriv', 'openssl', 'pillar',
  'pkgconfig', 'plyr', 'prettyunits', 'processx', 'prodlim', 'progress',
  'promises', 'ps', 'purrr', 'quantmod', 'R6', 'Rcpp', 'recipes', 'repr',
  'reprex', 'reshape2', 'rlang', 'rmarkdown', 'rstudioapi', 'rvest', 'scales',
  'selectr', 'shiny', 'SQUAREM', 'stringi', 'survival', 'sys', 'tibble',
  'tidyr', 'tidysele

In [3]:
BiocManager::install("rhdf5")

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'rhdf5'


package 'rhdf5' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\abraks\AppData\Local\Temp\RtmpS4FTES\downloaded_packages


Old packages: 'askpass', 'backports', 'BH', 'boot', 'broom', 'callr', 'caret',
  'class', 'cli', 'clipr', 'cluster', 'curl', 'data.table', 'DBI', 'dbplyr',
  'digest', 'dplyr', 'ellipsis', 'evaluate', 'fansi', 'forcats', 'foreach',
  'foreign', 'formatR', 'fs', 'ggplot2', 'glmnet', 'glue', 'gower', 'haven',
  'hexbin', 'hms', 'htmltools', 'htmlwidgets', 'httpuv', 'httr', 'ipred',
  'IRkernel', 'iterators', 'jsonlite', 'KernSmooth', 'knitr', 'later',
  'lattice', 'lava', 'lubridate', 'markdown', 'MASS', 'Matrix', 'mgcv', 'mime',
  'ModelMetrics', 'modelr', 'nlme', 'nnet', 'numDeriv', 'openssl', 'pillar',
  'pkgconfig', 'plyr', 'prettyunits', 'processx', 'prodlim', 'progress',
  'promises', 'ps', 'purrr', 'quantmod', 'R6', 'Rcpp', 'recipes', 'repr',
  'reprex', 'reshape2', 'rlang', 'rmarkdown', 'rstudioapi', 'rvest', 'scales',
  'selectr', 'shiny', 'SQUAREM', 'stringi', 'survival', 'sys', 'tibble',
  'tidyr', 'tidyselect', 'tidyverse', 'tinytex', 'TTR', 'uuid', 'whisker',
  'xfun', 'xml2

In [2]:
library("rhdf5")

Download the data (can take about 20-30 min)

In [5]:
destination_file = filepath
extracted_expression_file = "example_expression_matrix.tsv"

# Check if gene expression file was already downloaded, if not in current directory download file form repository
if(!file.exists(destination_file)){
    print("Downloading compressed gene expression matrix.")
    url = "https://s3.amazonaws.com/mssm-seq-matrix/human_matrix.h5"
    download.file(url, destination_file, quiet = FALSE, mode = "wb")
} else{
    print("Local file already exists.")
}

# Selected samples to be extracted
samp = c("GSM1224927","GSM1066120","GSM1224923","GSM1224929","GSM1224924","GSM1066118","GSM1066119","GSM1224925","GSM1224930","GSM1872071","GSM2282084","GSM1872064","GSM1872067","GSM1704845")

# Retrieve information from compressed data
samples = h5read(destination_file, "meta/Sample_geo_accession")
# Identify columns to be extracted
sample_locations = which(samples %in% samp)

tissue = h5read(destination_file, "meta/Sample_source_name_ch1")
genes = h5read(destination_file, "meta/genes")
series = h5read(destination_file, "meta/Sample_series_id")

# extract gene expression from compressed data
expression = h5read(destination_file, "data/expression", index=list(1:length(genes), sample_locations))
H5close()
rownames(expression) = genes
colnames(expression) = samples[sample_locations]

# Print file
write.table(expression, file=extracted_expression_file, sep="\t", quote=FALSE)
print(paste0("Expression file was created at ", getwd(), "/", extracted_expression_file))

[1] "Local file already exists."
[1] "Expression file was created at C:/Users/abraks/Documents/CS 548/ARCH5/example_expression_matrix.tsv"


How to read the data

In [7]:
gene_names = h5read(filepath, "meta/gene_name")
genes = h5read(filepath, "meta/genes")
gene_names[1:10]


[1] "alpha-1-B glycoprotein"                         
 [2] "APOBEC1 complementation factor"                 
 [3] "alpha-2-macroglobulin"                          
 [4] "alpha-2-macroglobulin like 1"                   
 [5] "alpha-2-macroglobulin pseudogene 1"             
 [6] "alpha 1,3-galactosyltransferase 2"              
 [7] "alpha 1,4-galactosyltransferase (P blood group)"
 [8] "alpha-1,4-N-acetylglucosaminyltransferase"      
 [9] "aladin WD repeat nucleoporin"                   
[10] "acetoacetyl-CoA synthetase"

In [8]:
sample_meta = h5read(filepath, "meta/Sample_title")


Below, you can see all of the keys to the dataset. Access each value by looking at the two cells above and changing the string. Most of the good stuff is under "meta/". The matrix is under "data/expression"

In [9]:
h5ls(filepath)

,group,name,otype,dclass,dim
0,/,data,H5I_GROUP,,
1,/data,expression,H5I_DATASET,INTEGER,35238 x 238522
2,/,info,H5I_GROUP,,
3,/info,author,H5I_DATASET,STRING,1
4,/info,contact,H5I_DATASET,STRING,1
5,/info,creation-date,H5I_DATASET,STRING,1
6,/info,lab,H5I_DATASET,STRING,1
7,/info,version,H5I_DATASET,STRING,1
8,/,meta,H5I_GROUP,,
9,/meta,Sample_channel_count,H5I_DATASET,STRING,238522


A preview of the expression matrix

In [11]:
expression = h5read(filepath, "data/expression", index = list(1:100,1:20))
expression

124,0,0,0,109,0,0,0,0,0,0,0,1113,0,86,0,1385,0,0,2741
77,0,0,0,56,0,0,0,0,0,0,0,9,0,57,0,3,0,0,10
10742,189,1518,1873,26007,835,1077,6362,545,1100,1208,2445,0,2315,12425,0,0,1770,361,0
27,6,0,0,20,0,0,0,0,0,5,0,4,0,19,0,4,0,0,9
119,0,0,0,70,0,0,0,0,0,0,0,0,0,64,0,0,0,0,8
25,0,0,0,38,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0
2328,0,0,0,2374,0,0,0,0,0,0,0,0,0,2471,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2838,0,0,0,2983,0,0,0,0,0,0,0,12,0,2972,0,46,0,0,38
2789,9,14,17,3067,15,29,1,14,17,21,0,226,13,2770,7,246,9,4,295
12,0,0,0,22,0,0,0,0,0,5,0,0,0,16,0,0,0,0,0


IMPORTANT: Close all h5 files when you're done with this command. 

In [13]:
h5closeAll()